In [2]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pymysql

In [16]:
conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234',
                       db='kbo_data_db', charset='utf8')

In [17]:
curs = conn.cursor(pymysql.cursors.DictCursor)

In [18]:
sql = '''
select player_id
,group_concat(player_year SEPARATOR ',') as year 
from kbo_pitcher_total_status
group by  player_id;
'''

In [19]:
curs.execute(sql)

183

In [20]:
rows = curs.fetchall()

In [21]:
play_info = {}
for row in rows:
    play_info[row['player_id']] = row['year'].split(',')
len(play_info)

183

In [22]:
driver = webdriver.Chrome('E:/BigData/BigData/driver/chromedriver.exe')
print('='*30,'start','='*30)
now = time.localtime()
print(now.tm_hour,'-',now.tm_min, '-',now.tm_sec)
err_sql=[]
for player_id,years in play_info.items():
    try:
        sql=''
        url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Daily.aspx?playerId={}'.format(player_id)
        driver.get(url)
        time.sleep(1)
        for year in years:
            select = Select(driver.find_element_by_id('cphContents_cphContents_cphContents_ddlYear'))
            select.select_by_value(str(year))
            time.sleep(1)
            html = driver.page_source
            bs = BeautifulSoup(html,'html.parser')
            trs = bs.select('#contents > div.sub-content > div.player_records > div > table > tbody > tr')
            for tr in trs:
                data_list = []
                tds = tr.select('td')
                for td in tds:
                    data_list.append(td.text)
                month_day = data_list[0]
                month = month_day.split('.')[0]
                day = month_day.split('.')[1]
                sql = '''
                insert into kbo_pitcher_daily_status values({},'{}','{}','{}','{}','{}','{}','{}','{}'
                ,'{}','{}','{}','{}','{}','{}','{}','{}','{}');
                '''.format(player_id,year,month,day,data_list[1],data_list[2],data_list[3],data_list[4]
                           ,data_list[5],data_list[6],data_list[7],data_list[8]
                           ,data_list[9],data_list[10],data_list[11],data_list[12]
                           ,data_list[13],data_list[14])
                curs.execute(sql)
            time.sleep(1)
    except:
        err_sql.append(sql)
        pass
print('='*30,'종료','='*30)
print(now.tm_hour,'-',now.tm_min,'-',now.tm_sec)

============================== start ==============================
13 - 5 - 4
============================== 종료 ==============================
13 - 5 - 4


In [13]:
driver = webdriver.Chrome('E:/BigData/BigData/driver/chromedriver.exe')
print('='*30,'시작','='*30)
now = time.localtime()
print(now.tm_hour,'-',now.tm_min, '-',now.tm_sec)
# try:
sql=''
url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Daily.aspx?playerId={}'.format(65933)
driver.get(url)
time.sleep(1)
for year in (2019,2018,2017):
    print(year)
    select = Select(driver.find_element_by_id('cphContents_cphContents_cphContents_ddlYear'))
    select.select_by_value(str(year))
    time.sleep(1)
    html = driver.page_source
    bs = BeautifulSoup(html,'html.parser')
    trs = bs.select('#contents > div.sub-content > div.player_records > div > table > tbody > tr')
    for tr in trs:
        data_list = []
        tds = tr.select('td')
        for td in tds:
            data_list.append(td.text)
        month_day = data_list[0]
        month = month_day.split('.')[0]
        day = month_day.split('.')[1]
        sql = '''
        insert into kbo_pitcher_daily_status values({},'{}','{}','{}','{}','{}','{}','{}','{}'
        ,'{}','{}','{}','{}','{}','{}','{}','{}','{}');
        '''.format(65933,year,month,day,data_list[1],data_list[2],data_list[3],data_list[4]
                   ,data_list[5],data_list[6],data_list[7],data_list[8]
                   ,data_list[9],data_list[10],data_list[11],data_list[12]
                   ,data_list[13],data_list[14])

    #         curs.execute(sql)
    time.sleep(1)
# except:
#     print(sql)
#     pass
print('='*30,'종료','='*30)
print(now.tm_hour,'-',now.tm_min, '-',now.tm_sec)

============================== 시작 ==============================
13 - 3 - 27
2019
2018
2017
============================== 종료 ==============================
13 - 3 - 27


In [24]:
err_sql

[]

In [23]:
conn.commit()